In [2]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras import layers,models

In [4]:
dataset_path="C:\\Users\\HP\\Downloads\\plant_disease"

In [90]:
dataset=tf.keras.preprocessing.image_dataset_from_directory(
    dataset_path,
    labels='inferred',   
    label_mode='int',
    shuffle=True,
    image_size=(224,224),
    batch_size=32
)   

Found 1532 files belonging to 3 classes.


In [92]:
class_names=dataset.class_names
class_names

['Test', 'Train', 'Validation']

In [94]:
def get_partition(dataset,train_split=0.8,test_split=0.1,val_split=0.1,shuffle=True,shuffle_size=1000):
    ds_size=len(dataset)
    if shuffle:
        ds=dataset.shuffle(shuffle_size,seed=12)
    train_size=int(train_split*ds_size)
    val_size=int(val_split*ds_size)
    train_ds=dataset.take(train_size)
    val_ds=dataset.skip(train_size).take(val_size)
    test_ds=dataset.skip(train_size).skip(val_size)

    return train_ds,val_ds,test_ds

In [96]:
train_ds,val_ds,test_ds=get_partition(dataset)

In [134]:
data_agumentation=tf.keras.Sequential([
    layers.RandomFlip('horizontal'),
    layers.RandomRotation(0.2),
    layers.RandomZoom(0.2),
    layers.RandomWidth(0.2),
    layers.RandomHeight(0.2)
])

In [144]:
from tensorflow.keras.applications import VGG16,ResNet50
from tensorflow.keras.layers import Flatten,Dense,Dropout
from tensorflow.keras import Model
from tensorflow.keras.regularizers import l2
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False

inputs=tf.keras.Input(shape=(224,224,3))
x=data_agumentation(inputs)
x = Flatten()(base_model.output)
x = Dense(64, activation='relu',kernel_regularizer=l2(0.01))(x)
x=Dropout(0.5)(x)
output = Dense(3, activation='softmax',kernel_regularizer=l2(0.01))(x)

# Create the final model
model = Model(inputs=base_model.input, outputs=output)

# Compile the model
model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy'])

# Summary of the model
model.summary()

Model: "functional_16"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_29 (InputLayer)     │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, 224, 224, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 224, 224, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 112, 112, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 112, 112, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 112, 112, 128)  │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 56, 56, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 56, 56, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, 28, 28, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, 28, 28, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, 14, 14, 512)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_pool (MaxPooling2D)      │ (None, 7, 7, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_19 (Flatten)            │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_36 (Dense)                │ (None, 64)             │     1,605,696 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_37 (Dense)                │ (None, 3)              │           195 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 16,320,579 (62.26 MB)

 Trainable params: 1,605,891 (6.13 MB)

 Non-trainable params: 14,714,688 (56.13 MB)

In [146]:
history=model.fit(train_ds,validation_data=val_ds,epochs=10)

Epoch 1/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 311s 8s/step - accuracy: 0.6786 - loss: 10.3591 - val_accuracy: 0.8125 - val_loss: 2.4198
Epoch 2/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 322s 8s/step - accuracy: 0.7839 - loss: 2.0671 - val_accuracy: 0.8750 - val_loss: 1.4882
Epoch 3/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 315s 8s/step - accuracy: 0.8538 - loss: 1.5188 - val_accuracy: 0.8359 - val_loss: 1.6344
Epoch 4/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 353s 9s/step - accuracy: 0.8842 - loss: 1.3505 - val_accuracy: 0.8516 - val_loss: 1.3854
Epoch 5/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 439s 12s/step - accuracy: 0.8887 - loss: 1.2399 - val_accuracy: 0.8672 - val_loss: 1.3702
Epoch 6/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 315s 8s/step - accuracy: 0.8881 - loss: 1.1856 - val_accuracy: 0.8828 - val_loss: 1.4067
Epoch 7/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 296s 8s/step - accuracy: 0.8890 - loss: 1.1575 - val_accuracy: 0.8906 - val_loss: 1.4222
Epoch 8/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 293s 8s/step - accuracy: 0.9074 - loss: 1.0700 - val_accuracy: 0.8516 -

In [148]:
model.evaluate(test_ds)

6/6 ━━━━━━━━━━━━━━━━━━━━ 53s 6s/step - accuracy: 0.8845 - loss: 1.2172


[1.2826201915740967, 0.8776595592498779]

In [156]:
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False

inputs=tf.keras.Input(shape=(224,224,3))
x=data_agumentation(inputs)
x=base_model(x,training=False)
x = Flatten()(base_model.output)
x = Dense(64, activation='relu')(x)
output = Dense(3, activation='softmax')(x)  # Adjust the number of classes

# Create the final model
model1= Model(inputs=base_model.input, outputs=output)

# Compile the model
model1.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy'])

# Summary of the model
model1.summary()

Model: "functional_18"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_33      │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_pad           │ (None, 230, 230,  │          0 │ input_layer_33[0… │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_conv (Conv2D) │ (None, 112, 112,  │      9,472 │ conv1_pad[0][0]   │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_bn            │ (None, 112, 112,  │        256 │ conv1_conv[0][0]  │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_relu          │ (None, 112, 112,  │          0 │ conv1_bn[0][0]    │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pad           │ (None, 114, 114,  │          0 │ conv1_relu[0][0]  │
│ (ZeroPadding2D)     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pool          │ (None, 56, 56,    │          0 │ pool1_pad[0][0]   │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_conv │ (None, 56, 56,    │      4,160 │ pool1_pool[0][0]  │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_bn   │ (None, 56, 56,    │        256 │ conv2_block1_1_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_relu │ (None, 56, 56,    │          0 │ conv2_block1_1_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_conv │ (None, 56, 56,    │     36,928 │ conv2_block1_1_r… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_bn   │ (None, 56, 56,    │        256 │ conv2_block1_2_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_relu │ (None, 56, 56,    │          0 │ conv2_block1_2_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_conv │ (None, 56, 56,    │     16,640 │ pool1_pool[0][0]  │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_conv │ (None, 56, 56,    │     16,640 │ conv2_block1_2_r… │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_bn   │ (None, 56, 56,    │      1,024 │ conv2_block1_0_c… │
│ (BatchNormalizatio… │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_bn   │ (None, 56, 56,    │      1,024 │ conv2_block1_3_c

 Total params: 30,010,499 (114.48 MB)

 Trainable params: 6,422,787 (24.50 MB)

 Non-trainable params: 23,587,712 (89.98 MB)

In [130]:
model1.fit(train_ds,validation_data=val_ds,epochs=10)

Epoch 1/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 152s 4s/step - accuracy: 0.7202 - loss: 2.1072 - val_accuracy: 0.8672 - val_loss: 0.4635
Epoch 2/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 162s 4s/step - accuracy: 0.8787 - loss: 0.3498 - val_accuracy: 0.8828 - val_loss: 0.4286
Epoch 3/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 169s 4s/step - accuracy: 0.9121 - loss: 0.2109 - val_accuracy: 0.8906 - val_loss: 0.3964
Epoch 4/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 154s 4s/step - accuracy: 0.9529 - loss: 0.1343 - val_accuracy: 0.8750 - val_loss: 0.4614
Epoch 5/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 154s 4s/step - accuracy: 0.9508 - loss: 0.1151 - val_accuracy: 0.9297 - val_loss: 0.2241
Epoch 6/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 179s 5s/step - accuracy: 0.9670 - loss: 0.0895 - val_accuracy: 0.9141 - val_loss: 0.3284
Epoch 7/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 187s 5s/step - accuracy: 0.9777 - loss: 0.0668 - val_accuracy: 0.9375 - val_loss: 0.2729
Epoch 8/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 177s 5s/step - accuracy: 0.9665 - loss: 0.0683 - val_accuracy: 0.9297 - v

In [132]:
model1.evaluate(test_ds)

6/6 ━━━━━━━━━━━━━━━━━━━━ 30s 2s/step - accuracy: 0.9228 - loss: 0.4057


[0.3524572551250458, 0.9255319237709045]

In [163]:
from tensorflow.keras.applications import InceptionResNetV2
base_model = InceptionResNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False

inputs=tf.keras.Input(shape=(224,224,3))
x=data_agumentation(inputs)
x=base_model(x,training=False)
x = Flatten()(base_model.output)
x = Dense(64, activation='relu',kernel_regularizer=l2(0.01))(x)
x=Dropout(0.5)(x)
output = Dense(3, activation='softmax',kernel_regularizer=l2(0.01))(x)  # Adjust the number of classes

# Create the final model
model2= Model(inputs=base_model.input, outputs=output)

# Compile the model
model2.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy'])

# Summary of the model
model2.summary()

Model: "functional_20"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_37      │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_406 (Conv2D) │ (None, 111, 111,  │        864 │ input_layer_37[0… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 111, 111,  │         96 │ conv2d_406[0][0]  │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_406      │ (None, 111, 111,  │          0 │ batch_normalizat… │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_407 (Conv2D) │ (None, 109, 109,  │      9,216 │ activation_406[0… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 109, 109,  │         96 │ conv2d_407[0][0]  │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_407      │ (None, 109, 109,  │          0 │ batch_normalizat… │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_408 (Conv2D) │ (None, 109, 109,  │     18,432 │ activation_407[0… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 109, 109,  │        192 │ conv2d_408[0][0]  │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_408      │ (None, 109, 109,  │          0 │ batch_normalizat… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_8     │ (None, 54, 54,    │          0 │ activation_408[0… │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_409 (Conv2D) │ (None, 54, 54,    │      5,120 │ max_pooling2d_8[… │
│                     │ 80)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 54, 54,    │        240 │ conv2d_409[0][0]  │
│ (BatchNormalizatio… │ 80)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_409      │ (None, 54, 54,    │          0 │ batch_normalizat… │
│ (Activation)        │ 80)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_410 (Conv2D) │ (None, 52, 52,    │    138,240 │ activation_409[0… │
│                     │ 192)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 52, 52,    │        576 │ conv2d_410[0][0]  │
│ (BatchNormalizatio… │ 192)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_410      │ (None, 52, 52,    │          0 │ batch_normalizat

 Total params: 56,794,595 (216.65 MB)

 Trainable params: 2,457,859 (9.38 MB)

 Non-trainable params: 54,336,736 (207.28 MB)

In [165]:
model2.fit(train_ds,validation_data=val_ds,epochs=10)

Epoch 1/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 222s 5s/step - accuracy: 0.7291 - loss: 66.4202 - val_accuracy: 0.8750 - val_loss: 2.1069
Epoch 2/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 182s 5s/step - accuracy: 0.8564 - loss: 2.4804 - val_accuracy: 0.8047 - val_loss: 1.9604
Epoch 3/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 179s 5s/step - accuracy: 0.8574 - loss: 2.0580 - val_accuracy: 0.8438 - val_loss: 1.9153
Epoch 4/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 206s 5s/step - accuracy: 0.8602 - loss: 1.9093 - val_accuracy: 0.8750 - val_loss: 1.8959
Epoch 5/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 226s 6s/step - accuracy: 0.8638 - loss: 2.8852 - val_accuracy: 0.8438 - val_loss: 1.8865
Epoch 6/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 234s 6s/step - accuracy: 0.8427 - loss: 1.8845 - val_accuracy: 0.8047 - val_loss: 1.8795
Epoch 7/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 180s 5s/step - accuracy: 0.8620 - loss: 1.8747 - val_accuracy: 0.7578 - val_loss: 1.8735
Epoch 8/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 176s 5s/step - accuracy: 0.8538 - loss: 1.8663 - val_accuracy: 0.8203 - 

In [169]:
model2.evaluate(test_ds)

6/6 ━━━━━━━━━━━━━━━━━━━━ 36s 3s/step - accuracy: 0.8856 - loss: 1.8407


[1.8404804468154907, 0.8882978558540344]